# Crawling - 네이버 뷰 '영화'

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
chromedriver = '/Users/정현/Desktop/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [3]:
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='
url = base_url + quote('영화')

In [6]:
driver.get(url)

In [8]:
# blog 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for i in range(20):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [9]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [10]:
lis = soup.select('li.bx._svp_item')
len(lis)

45

In [11]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
date = li.select_one('.sub_time.sub_txt').get_text()
title, date, content

('넷플릭스 로맨틱 코미디 영화 추천 10편 소개해요',
 '2021.08.13.',
 '넷플릭스 로맨틱 코미디 영화 추천 10편 소개해요 오늘은 금요일. 불금이 될 것 같지는 않고 늘 그렇듯... 오늘은 그중에서도 사람들의 선호도가 높은 로맨틱 코미디 영화 목록이다. 총 열개를 선정해보았고 뽑은건 전부...')

In [12]:
title_list, content_list, date_list = [], [], []
for i, li in enumerate(lis):
    title = li.select_one('a.api_txt_lines.total_tit').get_text()
    content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
    date = li.select_one('.sub_time.sub_txt').get_text()

    title_list.append(title)
    content_list.append(content)
    date_list.append(date)

    # print(i)

In [13]:
df = pd.DataFrame({
    'title':title_list, 'date':date_list, 'content':content_list
})
df.head()

,title,date,content
0,넷플릭스 로맨틱 코미디 영화 추천 10편 소개해요,2021.08.13.,넷플릭스 로맨틱 코미디 영화 추천 10편 소개해요 오늘은 금요일. 불금이 될 것 같...
1,추석엔 한국 영화라는 공식은?_9월 개봉 예정 영화 추천 9 [+1 넷플릭스 신작],어제,추석엔 한국 영화라는 공식은?_9월 개봉 예정 영화 9 진짜 9월이 바로 코앞입니다...
2,"넷플릭스 19영화 추천-주말 밤, 몰래봐야 하는 파격적인 미친 사랑을 다룬 청불 영...",2021.08.21.,"오늘은 장마 기간 집에서 혼자, 혹은 연인들이 볼만한 넷플릭스 청불 영화 몇 편에 ..."
3,넷플릭스 오리지널 영화 스위트 걸 리뷰 * 제목이 스포였어!,2021.08.22.,이럴 땐 집중력을 요하는 책 읽기보단 멍 때리며 볼 수 있는 영화가 낫지! 역시나 ...
4,"이게 뉴스야, 영화야? 극 현실주의라 더 무서운 국내 재난 영화",2021.08.11.,"재난 영화에서나 일어날 법한 일이 현실이 되고, 그것이 일상이 되어버린 요즘. 또 ..."


In [14]:
df.shape

(45, 3)

In [15]:
df.to_csv('data/movie.txt')

In [16]:
driver.close()